## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Sibolga,ID,1.7427,98.7792,78.58,85,99,2.37,overcast clouds
1,1,Aksu,CN,41.1231,80.2644,73.29,20,91,4.63,overcast clouds
2,2,Kushmurun,KZ,52.4503,64.6311,52.97,50,67,10.49,broken clouds
3,3,Victoria,HK,22.2855,114.1577,79.00,85,77,1.01,broken clouds
4,4,Asau,RO,46.4333,26.4000,62.01,53,92,3.40,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Sibolga,ID,1.7427,98.7792,78.58,85,99,2.37,overcast clouds
1,1,Aksu,CN,41.1231,80.2644,73.29,20,91,4.63,overcast clouds
3,3,Victoria,HK,22.2855,114.1577,79.00,85,77,1.01,broken clouds
4,4,Asau,RO,46.4333,26.4000,62.01,53,92,3.40,overcast clouds
6,6,Concordia,AR,-31.3930,-58.0209,79.00,38,0,11.50,clear sky
9,9,Rikitea,PF,-23.1203,-134.9692,76.48,79,88,11.03,light rain
12,12,Platanos,GR,37.6667,21.6167,72.00,56,40,5.97,scattered clouds
13,13,Kavieng,PG,-2.5744,150.7967,81.50,76,85,3.91,overcast clouds
14,14,Puerto Ayora,EC,-0.7393,-90.3518,82.40,65,40,20.71,scattered clouds
15,15,Hilo,US,19.7297,-155.0900,77.00,65,40,3.44,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                392
City                   392
Country                388
Lat                    392
Lng                    392
Max Temp               392
Humidity               392
Cloudiness             392
Wind Speed             392
Current Description    392
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
new_preferred_cities_df = preferred_cities_df.dropna()

In [7]:
new_preferred_cities_df.count()

City_ID                388
City                   388
Country                388
Lat                    388
Lng                    388
Max Temp               388
Humidity               388
Cloudiness             388
Wind Speed             388
Current Description    388
dtype: int64

In [8]:
new_preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Sibolga,ID,1.7427,98.7792,78.58,85,99,2.37,overcast clouds
1,1,Aksu,CN,41.1231,80.2644,73.29,20,91,4.63,overcast clouds
3,3,Victoria,HK,22.2855,114.1577,79.00,85,77,1.01,broken clouds
4,4,Asau,RO,46.4333,26.4000,62.01,53,92,3.40,overcast clouds
6,6,Concordia,AR,-31.3930,-58.0209,79.00,38,0,11.50,clear sky


In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = new_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Sibolga,ID,78.58,overcast clouds,1.7427,98.7792,
1,Aksu,CN,73.29,overcast clouds,41.1231,80.2644,
3,Victoria,HK,79.00,broken clouds,22.2855,114.1577,
4,Asau,RO,62.01,overcast clouds,46.4333,26.4000,
6,Concordia,AR,79.00,clear sky,-31.3930,-58.0209,
9,Rikitea,PF,76.48,light rain,-23.1203,-134.9692,
12,Platanos,GR,72.00,scattered clouds,37.6667,21.6167,
13,Kavieng,PG,81.50,overcast clouds,-2.5744,150.7967,
14,Puerto Ayora,EC,82.40,scattered clouds,-0.7393,-90.3518,
15,Hilo,US,77.00,light rain,19.7297,-155.0900,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
nan_value = float("NaN")
hotel_df.replace("", nan_value, inplace=True)
hotel_df.dropna(subset = ["Hotel Name"], inplace=True)
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Sibolga,ID,78.58,overcast clouds,1.7427,98.7792,Hotel Syariah CN Darussalam
1,Aksu,CN,73.29,overcast clouds,41.1231,80.2644,Pudong Holiday Hotel
3,Victoria,HK,79.00,broken clouds,22.2855,114.1577,Mini Hotel Central
4,Asau,RO,62.01,overcast clouds,46.4333,26.4000,Casa Mika Ciobanus
6,Concordia,AR,79.00,clear sky,-31.3930,-58.0209,Palmar Hotel Casino
...,...,...,...,...,...,...,...
679,Redding,US,73.99,clear sky,40.5865,-122.3917,Redding Inn & Suites
681,Namanyere,TZ,61.50,light rain,-7.5167,31.0500,Manyovu Lodge
682,Bima,ID,75.02,overcast clouds,-8.4667,118.7167,Lambitu Hotel
684,Xichang,CN,61.41,overcast clouds,27.8964,102.2634,Xichang ibis Hotel


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))